In [1]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

In [2]:
from lib import *

In [3]:
def write_stations(list_temperature, list_precipitation, list_streamflow, pathout):
    mkNestedDir(pathout)

    line_1 = "{num_temperature}                 ! number of T stations"
    line_2 = "{num_precipitation}               ! number of P stations"
    line_3 = "{num_streamflow}                  ! number of Q stations"
    line_T = "! T stations ID"
    line_P = "! P stations ID"
    line_Q = "! Q stations ID"

    line_1 = line_1.format(num_temperature=str(len(list_temperature)))
    line_2 = line_2.format(num_precipitation=str(len(list_precipitation)))
    line_3 = line_3.format(num_streamflow=str(len(list_streamflow)))

    file = open(pathout+"stations.txt","w+")
    file.writelines(line_1 + "\n")
    file.writelines(line_2 + "\n")
    file.writelines(line_3 + "\n")

    file.writelines(line_T + "\n")
    for t in list_temperature:
        file.writelines(str(t) + "\n")

    file.writelines(line_P + "\n")
    for p in list_precipitation:
        file.writelines(str(p) + "\n")

    file.writelines(line_Q + "\n")
    for q in list_streamflow:
        file.writelines(str(q) + "\n")

In [4]:
def write_configuration_file(run_id, run_mode, start_wu_date, end_wu_date, start_forecast_date, end_forecast_date,
                             parameters_file, basin, meteo_src, outlet_point, da_flag, wdir, pathout, kernel=True):
    mkNestedDir(pathout)

    if kernel != True:
        wwdir = wdir
        wdir = '../'
    else:
        wwdir = wdir

    line_1 = "{run_mode}                                ! run mode: 10 -> forecasting; 6 -> forward".format(run_mode=run_mode)
    line_2 = "{run_id}                                         ! ID run".format(run_id=run_id)
    line_3 = "0 				 		                ! ID previous run (useless)"
    line_4 = "100 			 		                    ! ID run meteorological fields (useless)"
    line_5 = "0  	 			 		                ! T forecasts correction: 1 -> yes; 0 -> no (useless)"
    line_6 = "0 				 		                ! T forecasts correction: 1 -> yes; 0 -> no (useless)"

    # warmup period
    line_9 = "{start_wu_date} 		                    ! start warm - up".format(
        start_wu_date=dt.datetime.strftime(start_wu_date, format='%Y-%m-%d_%H:%M'))
    line_10 = "{end_wu_date} 		                    ! begin D.A. (end warm-up)".format(
        end_wu_date=dt.datetime.strftime(end_wu_date, format='%Y-%m-%d_%H:%M'))

    # DA starts one hour later than line_10 and it ends at line_7

    # forecast period
    line_7 = "{start_forecast_date}                     ! end D.A. (start forecast/forward)".format(
        start_forecast_date=dt.datetime.strftime(start_forecast_date, format='%Y-%m-%d_%H:%M'))
    line_8 = "{end_forecast_date} 	                    ! end forecast/forward".format(
        end_forecast_date=dt.datetime.strftime(end_forecast_date, format='%Y-%m-%d_%H:%M'))

    # parameters to run the model
    line_11 = "\"{parameters_file}\" 	                    ! parameter file name".format(
        parameters_file=parameters_file)

    line_12 = "\"subcatchments.asc\"                     ! subcatchment raster file name"
    line_13 = "\"{wdir}{basin}/INPUT/\" 	                ! path of required input data".format(
        wdir=wdir, basin=basin)
    
    line_14 = "\"{wdir}{basin}/OUTPUT/{date}/\" 	         ! general folder of output data"
    if run_mode == '10':
        line_14 = line_14.format(wdir=wdir, basin=basin, date=dt.datetime.strftime(start_forecast_date, format='%Y%m%d'))
        out_path = "{wdir}{basin}/OUTPUT/{date}/".format(wdir=wwdir, basin=basin, date=dt.datetime.strftime(start_forecast_date, format='%Y%m%d'))
    else:
        line_14 = line_14.format(wdir=wdir, basin=basin, date='')
        out_path = "{wdir}{basin}/OUTPUT/".format(wdir=wwdir, basin=basin)

    mkNestedDir(out_path)

    line_15 = "\"{wdir}{basin}/meteo/\"                     ! path of required meteorological data (obs. and for.)".format(
        wdir=wdir, basin=basin)
    line_16 = "\"{meteo_src}\"			                ! observed meteo source".format(
        meteo_src=meteo_src)
    line_17 = "30.0					                    ! cellsize of subcatchment map"
    line_18 = "5.0						                ! mm SWE for SCA threshold"
    line_19 = "46.7 					                    ! mean latitude"
    line_20 = "2.0						                ! minimum number of meteorological station to perform interpolation"
    line_21 = "\"NASH\" 					                ! objective function for model calibration"
    line_22 = "1 						                ! number of calibration points"
    line_23 = "{outlet_point} 					        ! ID calibration point".format(
        outlet_point=outlet_point)
    line_24 = "{outlet_point} 					        ! ID outlet section".format(
        outlet_point=outlet_point)
    line_25 = "1 						                ! model type for base"
    line_26 = "{da_flag} 						        ! flag Data Assimilation: 1 -> active; 0 -> open-loop".format(
        da_flag=da_flag)

    file = open(pathout+"configuration_file.txt", "w+")
    [file.writelines(l + "\n") for l in [line_1, line_2, line_3, line_4, line_5, line_6, line_7, line_8, line_9, line_10, line_11,
                                         line_12, line_13, line_14, line_15, line_16, line_17, line_18, line_19, line_20, line_21,
                                         line_22, line_23, line_24, line_25, line_26]]

In [5]:
def write_DA_settings(ensemble_size, pathout):
    mkNestedDir(pathout)

    line_1 = "{ensemble_size} 			! ensemble size".format(ensemble_size=ensemble_size)
    line_2 = "5 			            ! number of state variables"
    line_3 = "R C S G B 	            ! labels of state variables"
    line_4 = "0.00		                ! percentage error of observed Q "
    line_5 = "0.01			            ! percentage error for further perturbing observed streamflow"
    line_6 = "0.02		                ! threshold on precipitation perturbation"
    line_7 = "1.0			            ! shape gamma pdf for P perturbation (k or alpha)      --> USELESS"
    line_8 = "1.0			            ! scale gamma pdf for P perturbation (beta or 1/theta) --> USELESS"
    line_9 = "0. 			            ! dev. std. T perturbation"
    line_10 = "0			            ! additive inflation on soil water content (0: no; 1: yes)"
    line_11 = "0.00		                ! random noise for additive inflation"
    line_12 = "1	    	            ! time step between every analysis step"

    file = open( pathout + "DA_settings.txt", "w+")
    [file.writelines(l + "\n") for l in [line_1, line_2, line_3, line_4, line_5, line_6,
                                         line_7, line_8, line_9, line_10, line_11, line_12]]

In [6]:
def write_input(basin, pathout):
    mkNestedDir(pathout)

    line_1 = '''"../{basin}/INPUT/"'''.format(basin=basin)
    line_1 = line_1.format(basin=basin)

    file = open(pathout+"input_path.txt","w+")
    [file.writelines(el + "\n") for el in [line_1]]

## STEPS
1. setup the wdir and the basin name
2. setup the start and end date of simulation [both warm up and simulation]
3. setup the parameter file
4. setup the source of meteo data
5. setup the outlet point 

In [7]:
wdir = "/media/windows/projects/era5_bias/"
basin = "passirio"

start_wu_date_str = '2010-10-01 00:00:00'
end_wu_date_str = '2013-09-30 00:00:00'

start_date_str = '2013-10-01 00:00:00'
end_date_str = '2017-09-30 00:00:00'

timezone_str = 'Europe/Rome'

parameters_file = "hydro_parameters_rea_plan_4283.txt"
meteo_src = "reanalysis"
outlet_point = "155"

run_id = "10"

In [8]:
start_wu_date = dt.datetime.strptime(
    start_wu_date_str, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.timezone(timezone_str))
end_wu_date = dt.datetime.strptime(
    end_wu_date_str, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.timezone(timezone_str))

start_date = dt.datetime.strptime(
    start_date_str, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.timezone(timezone_str))
end_date = dt.datetime.strptime(
    end_date_str, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.timezone(timezone_str))

In [9]:
output_dt_format = '%Y-%m-%d %H:%M:%S'

# setup simulations
## forecasting => 10
## forward => 6
run_mode = "6"
da_flag = "0"
ensemble_size = 1

### ICHYMOD LINUX
linux = True
exe_path = "/home/daniele/documents/github/ftt01/phd/models/hydro_modeling/ichymod/core/release/"
exe_name = "ichymod"
exe_local_path = wdir + 'hydro_modeling/exe/'
mkNestedDir(exe_local_path)

In [10]:
write_configuration_file(
    run_id, run_mode, start_wu_date, end_wu_date, start_date, end_date, 
    parameters_file, basin, meteo_src, outlet_point, 
    da_flag, wdir + "hydro_modeling/", wdir + "hydro_modeling/" + basin + "/INPUT/", kernel=linux)

write_DA_settings(ensemble_size, wdir + "hydro_modeling/" + basin + "/INPUT/")

write_input( basin, exe_local_path )

### copy the exe in the directory
shutil.copyfile(
    exe_path + exe_name,
    exe_local_path + exe_name)

'/media/windows/projects/era5_bias/hydro_modeling/exe/ichymod'

In [11]:
### RUN THE MODEL
cmd = '''cd {path} && {exe_path}{exe_name}'''.format(path=exe_local_path, exe_path=exe_path, exe_name=exe_name)
print(cmd)

p = subprocess.Popen( cmd, stdin=subprocess.PIPE, shell=True)
# p.communicate(input='\n'.encode())

cd /media/windows/projects/era5_bias/hydro_modeling/exe/ && /home/daniele/documents/github/ftt01/phd/models/hydro_modeling/ichymod/core/release/ichymod
Time: 15:11:24


forrtl: severe (59): list-directed I/O syntax error, unit 1, file /media/windows/projects/era5_bias/hydro_modeling/exe/../passirio/INPUT/configuration_file.txt
Image              PC                Routine            Line        Source             
ichymod            000000000048D428  Unknown               Unknown  Unknown
ichymod            00000000004B6802  Unknown               Unknown  Unknown
ichymod            00000000004B5267  Unknown               Unknown  Unknown
ichymod            0000000000470747  Unknown               Unknown  Unknown
ichymod            000000000045F9E7  Unknown               Unknown  Unknown
ichymod            000000000046E9D5  Unknown               Unknown  Unknown
ichymod            0000000000403822  Unknown               Unknown  Unknown
libc.so.6          00007FA623429D90  Unknown               Unknown  Unknown
libc.so.6          00007FA623429E40  __libc_start_main     Unknown  Unknown
ichymod            000000000040372E  Unknown               Unknown  

In [ ]:
### read the output
